<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/PracticasDeDesarrollo/3_Desarrollo_III/Ejercicios/ejercicio_combinado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>

# Ejercicio de Buenas Prácticas de Desarrollo

En este ejercicio vamos a poner en práctica los siguientes conocimientos correspondientes a la Clase 3 del curso:

- Cookiecutter para estructurar proyectos
- PyLint
- Documentar nuestro código
- Crear paquete con `setuptools`
- Crear y utilizar entornos virtuales con Conda

# Empecemos

El código de abajo es una adaptación de un código de ejemplo que extraje de uno de mis proyectos viejos. Lo que hace es definir un _decorador_. Vimos un ejemplo de decorador en la Clase I del curso de Buenas Prácticas, con el uso del paquete `click`. La sintaxis `@click.command()`, seguida de una declaración de una función via la sintaxis `def funcion()...`, es un ejemplo de uso de un decorador.

Veamos el código:

In [3]:
%%writefile decorators.py
import functools
from typing import Dict, Iterable, Union
from numpy.random import choice, random

def anadir_ruido(noise_probability : float, noise_distribution : Dict[str, float]):
    def inner_decorator(func):
        possible_noise_values = list(noise_distribution.keys())
        noise_value_probabilities = list(noise_distribution.values())
        @functools.wraps(func)
        def wrapper(*args):
            true_value = func(*args)
            if random() <= noise_probability: #ie con prob `noise_probability`
                vals, probs = exclude_true_value(possible_noise_values, noise_value_probabilities, true_value)
                return choice(vals, p=probs)
            else:
                return true_value
        return wrapper
    return inner_decorator


def exclude_true_value(possible_noise_values, noise_value_probabilities, true_value):
    try:
        i = possible_noise_values.index(true_value)
        possible_noise_values = possible_noise_values[:i] + possible_noise_values[i+1:]
        noise_value_probabilities = noise_value_probabilities[:i] + noise_value_probabilities[i+1:]
        z = sum(noise_value_probabilities)
        noise_value_probabilities = [x/z for x in noise_value_probabilities]
    except ValueError:
        pass
    return possible_noise_values, noise_value_probabilities

if __name__ == "__main__":
    @anadir_ruido(0.5, {'ups!':0.7, 'UPS!':0.3})
    def funcion_de_ejemplo(x):
        return 2*x
    for x in range(10):
        print(funcion_de_ejemplo(x))
    

Overwriting decorators.py


En la parte abajo de `if __name__ == "__main__"` (que recordemos, solo es ejecutada si el código es llamado como script, y _no_ es ejecutada si el código es importado como módulo), tenemos un ejemplo de uso. Ejecutémoslo:

In [1]:
!python decorators.py

0
2
ups!
6
UPS!
10
ups!
14
16
18


La idea es que si queremos definir una función, posiblemente compleja, y al final pedirle que su resultado sea a veces reemplazado, aleatoriamente, por algún otro valor, esta última funcionalidad podemos agregarla con la línea `@anadir_ruido(0.5, {'ups!':0.7, 'UPS!':0.3})` antes de la definición de la función.

Más allá de los detalles de cómo funcionan los decoradores, la idea es usar este código como punto de partida para los siguientes ejercicios (no es necesario que sean expertos en decoradores para completarlos).

Esta es nuestra hoja de ruta:

- Comenzar un proyecto con `cookiecutter`, meter `decorators.py` dentro de ese proyecto.
- Chequear el código con PyLint, adaptarlo a las reglas PEP
- Documentar el código
- Publicar el código como un _paquete_, para ser instalado con pip desde github.
- Crear un entorno virtual, instalar nuestro paquete y usarlo

# Ejercicio


1. Crear un proyecto usando esta plantilla de Cookiecutter: `https://github.com/oldani/cookiecutter-simple-pypackage`

En algún momento a partir de ahora, será necesario inicializar un repositorio de git en la carpeta de nuestro proyecto, realizar el commit inicial y posteriormente commitear todo cambio adicional que querramos incorporar.

2. Incorporar el código de `decorators.py` en el proyecto (para pensar: ¿En qué carpeta/archivo?)
3. Ejecutar PyLint sobre nuestro código y corregir lo que nos diga. Entre otras cosas, será necesario escribir _docstrings_ para las funciones que incorporamos, y quizá para un módulo (si es que agregamos un nuevo archivo al proyecto).
4. Revisar el contenido de `README.md`y `setup.py`. No se preocupen por que todos los detalles estén puestos correctamente, en principio debería funcionar sin que lo toquen manualmente. Pero no toda la información generada automáticamente va a ser correcta. Por ejemplo, en el README, la instrucción de instalación no va a ser correcta, dado que no vamos a usar pypi para distribuir nuestro paquete. Tendríamos que reemplazar la línea `pip install mi_proyecto_de_prueba` en el README por otra que diga `pip install git+https://github.com/nuestro-user/mi_proyecto_de_prueba.git`.
5. Publicar en Github nuestro proyecto (asegurarse de usar el mismo nombre para el repo de github que el que elegimos antes).

Ahora vamos a usar nuestro proyecto, dentro de otro proyecto.

6. Crear un entorno con conda (`conda create --name nombre-de-mi-entorno`) y movernos a este entorno (`conda activate nombre-de-mi-entorno`).
7. Instalar nuestro paquete desde github (`pip install git+https://github.com/nuestro-user/mi_proyecto_de_prueba.git`)
8. Crear un script que use nuestro paquete y ejecutarlo.
   
Opcionalmente, luego podemos eliminar el entorno, para mantener limpio nuestro sistema (`conda env remove --name nombre-de-mi-entorno`).

### Extra (si hay tiempo y ganas)

Modificar nuestro paquete. Supongamos que queremos agregar algo más a nuestro paquete, por ejemplo una función.

9. Para hacerlo lo más sencillo posible, agreguemos una función `hola_mundo` (que imprime un saludo en pantalla) al `__init__.py` de nuestro paquete.
10. Modificamos el `setup.py` para que indique la nueva versión del paquete (por ejemplo, cambiando `version='0.1.0'` a `version='0.1.1'`). Recordar que esto es importante pues, de lo contrario, pip no va a saber que tiene que actualizar el paquete y no lo va a hacer, incluso si ejecutamos el comando de instalación nuevamente.
11. Pusheamos cambios al repo de github
12. Ejecutamos nuevamente la línea de instalación del paquete.
13. Ahora deberíamos poder llamar nuestra nueva función importándola según `from mi_proyecto_de_prueba import hola_mundo`.